In [ ]:
%cd ..

In [ ]:
from apperception.database import database

In [29]:
############ ROAD DIRECTION ############

In [ ]:
### Add Road Direction Attributes to Tables ###
query = """ALTER TABLE Item_General_Trajectory
           ADD roadDirection tfloat;

           ALTER TABLE Cameras
           ADD roadDirection numeric;
        """
database._execute_query(query=query)

In [ ]:
### Update Road Direction For Camera ###
query = """UPDATE Cameras
           SET roadDirection = roadDirection(egoTranslation, egoHeading)
           WHERE roadDirection IS NULL;
        """
database._execute_query(query=query)

In [61]:
### Create Function For roadDirection With Traj ###
query = """ 
        DROP FUNCTION IF EXISTS _roadDirectionAttr(trajectory tgeompoint, headings tfloat);
        CREATE OR REPLACE FUNCTION _roadDirectionAttr(trajectory tgeompoint, headings tfloat) RETURNS tfloat AS
        $BODY$
        declare trajString text;
        declare trajPointString text;
        declare trajPoint geometry;
        declare trajTime timestamptz;
        BEGIN
        trajString := asText(trajectory);
        trajString := substr(trajString, 2, length(trajString) - 2);
        FOREACH trajPointString IN ARRAY STRING_TO_ARRAY(trajString, ',')
        LOOP
                raise notice 'trajPoint: %', STRING_TO_ARRAY(trajPointString, '@');
                trajPoint := ST_GeomFromText((STRING_TO_ARRAY(trajPointString, '@'))[0]);
                trajTime :=  (STRING_TO_ARRAY(trajPointString, '@'))[2];
        END LOOP;
        END
        $BODY$
        LANGUAGE 'plpgsql' ;

        SELECT _roadDirectionAttr(tgeompoint  '[Point(1 1)@2001-01-01, Point(2 2)@2001-01-03, POINT(3 3)@2001-01-05]', tfloat  '[1@2001-01-01, 2@2001-01-03, 3@2001-01-05]')
        """
database._execute_query(query=query)
# SELECT STRING_TO_ARRAY(asText(tgeompoint  '[Point(1 1)@2001-01-01, Point(2 2)@2001-01-03, POINT(3 3)@2001-01-05]'), ',');


FunctionExecutedNoReturnStatement: control reached end of function without RETURN
CONTEXT:  PL/pgSQL function _roaddirectionattr(tgeompoint,tfloat)


In [30]:
############ MIN AND MAX TIMESTAMPS ############

In [26]:
### Add minTimestamp Attributes to Trajectory Table ###
query = """ALTER TABLE Item_General_Trajectory
           ADD minTimestamp timestamptz;

           ALTER TABLE Item_General_Trajectory
           ADD maxTimestamp timestamptz;
        """
database._execute_query(query=query)

In [27]:
### Update minTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET minTimestamp = lower(trajCentroids::period)
           WHERE minTimestamp IS NULL;
        """
database._execute_query(query=query)

In [28]:
### Update maxTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET maxTimestamp = upper(trajCentroids::period)
           WHERE maxTimestamp IS NULL;
        """
database._execute_query(query=query)

In [31]:
############ ROAD SEGMENT TYPES ############